# Introduction to FandangoLearner

This notebook demonstrates how to use **FandangoLearner**, a pattern based approach that automatically learns constraints to explain why a program fails.

The core idea of FandangoLearner is to identify patterns in inputs 
that lead to program errors or unexpected behaviors. Using these patterns, 
it generates constraints in the Fandango language to help developers 
understand input-related bugs.

## Step 1: Define the Grammar
We start by defining the grammar for our input language.
This example focuses on arithmetic expressions using trigonometric and square root functions.

In [1]:
from fdlearn.interface.fandango import parse_contents

grammar = """
<start> ::= <arithexp>;
<arithexp> ::= <function>"("<number>")";
<function> ::= "sqrt" | "cos" | "sin" | "tan";

<number> ::= <maybeminus><onenine><maybedigits> | "0";
<maybeminus> ::= "-" | "";
<onenine> ::= "1" | "2" | "3" | "4" | "5" | "6" | "7" | "8" | "9";
<maybedigits> ::= <digit>*;
<digit>::=  "0" | <onenine>;
"""

grammar, _ = parse_contents(grammar)

## Step 2: Provide Initial Inputs

We supply a set of example inputs along with their expected outcomes (`True` for failure, `False` otherwise).


In [2]:
initial_inputs = {
    ("sqrt(-900)", True),  # This input causes a failure.
    ("sqrt(-10)", True),   # Another failure case.
    ("sqrt(0)", False),    # This input works correctly.
    ("sin(-900)", False),  # Works correctly.
    ("sqrt(2)", False),    # Works correctly.
    ("cos(10)", False),    # Works correctly.
}

Convert inputs to FandangoInput objects

In [3]:
from fdlearn.learner import FandangoInput

initial_inputs = {
  FandangoInput.from_str(grammar, inp, oracle)
  for inp, oracle in initial_inputs
}

### Step 3: Select Relevant Non-Terminals (Optional)

We specify the non-terminals in the grammar that are likely related to the program's failure behavior.
This step is optional but can help focus the learning process on specific parts of the grammar.
Later, we will see that **Avicenna** can automatically learn relevant non-terminals. 


In [4]:
from fdlearn.learner import NonTerminal

relevant_non_terminals = {
    NonTerminal("<number>"),
    NonTerminal("<maybeminus>"),
    NonTerminal("<function>"),
}

## Step 4: Learn Constraints

Using the `FandangoLearner`, we learn constraints that explain why certain inputs fail.


In [5]:
from fdlearn.learner import FandangoLearner

learner = FandangoLearner(grammar)

learned_constraints = learner.learn_constraints(
    initial_inputs,
    relevant_non_terminals=relevant_non_terminals
)

fandango-learner:INFO: Instantiated patterns: 22
fandango-learner:INFO: Filtered positive inputs for learning: 2
fandango-learner:INFO: Evaluating 147 candidates
fandango-learner:INFO: Calculating combinations for 26 candidates
fandango-learner:INFO: Found 48 valid conjunctions
fandango-learner:INFO: Learned 9 constraint(s) that meet(s) the criteria


## Step 5: Analyze Results

Finally, we analyze the constraints generated by FandangoLearner to understand the root cause of failures.


In [6]:
for candidate in learner.get_best_candidates():
    print(candidate.constraint)

(int(<number>) <= -10 and str(<function>) == 'sqrt')
((exists <elem> in <maybeminus>: '-' in <elem>) and str(<function>) == 'sqrt')
((exists <elem> in <function>: str(<elem>) == 'sqrt') and (exists <elem> in <maybeminus>: str(<elem>) == '-'))
((exists <elem> in <function>: str(<elem>) == 'sqrt') and (exists <elem> in <maybeminus>: '-' in <elem>))
((exists <elem> in <function>: 'sqrt' in <elem>) and (exists <elem> in <maybeminus>: '-' in <elem>))
((exists <elem> in <function>: str(<elem>) == 'sqrt') and int(<number>) <= -10)
((exists <elem> in <maybeminus>: str(<elem>) == '-') and str(<function>) == 'sqrt')
((exists <elem> in <function>: 'sqrt' in <elem>) and (exists <elem> in <maybeminus>: str(<elem>) == '-'))
((exists <elem> in <function>: 'sqrt' in <elem>) and int(<number>) <= -10)


The output will show the constraints that best explain the failures in the initial inputs.

We can see that the constraint `(str(<function>) == 'sqrt' and int(<number>) <= -10)` explains why the inputs `sqrt(-900)` and `sqrt(-10)` fail.
However, this constraint is too specific and does not generalize well to other inputs.
Thus, we need a feedback loop that automatically refines these constraints to generate general constraints that captures the essence of the failure.
We will use **Avicenna** to provide this feedback loop.